# Comparing OLS and Double Selection
This material is originally developed for ACE 592 Big Data - Empirical Economics written on 7/21/2017 by Alex Stevens. 

### Motivation
Researchers do not know the data generating process for the data they are analyzing. This exercise creates a nonlinear data generating process that is too complex for simple modelling to capture, then compares OLS and Double Selection estimation.

### Objective:
* Create Monte Carlo experiement
* Compare distribution of coeffiecients estimated by OLS and Double Selection 
* Look at the effects of assumptions on these two models

Remember to shut down your server when you are done by clicking Control Panel -> Shut Down Server

## Step 1: Install packages 

In [ ]:
rm(list=ls())
## This function will check if a package is installed, and if not, install it
pkgTest <- function(x) {
  if (!require(x, character.only = TRUE))
  {
    install.packages(x, dep = TRUE, repos = "http://cran.us.r-project.org")
    if(!require(x, character.only = TRUE)) stop("Package not found")
  }
}

## These lines load the required packages
packages <- c("Hmisc", "hdm","mvtnorm")
lapply(packages, pkgTest)

## Step 2: Create data


Set seed for replication

In [ ]:
set.seed(12345)

Set sample size

In [ ]:
N <- 200

Set number of simulations

In [ ]:
n.sims <- 40

Create matrix of 200 random normally distributed variables

In [ ]:
x <- matrix(runif(N*50), nrow = N, ncol = 50)

Create polynomials

In [ ]:
polyx<-cbind(x^2,x^3)

Create 5 bins for first 40 variables

In [ ]:
nvar<-10
quants<-5
dumx<-matrix(nrow=N,ncol=quants*nvar)
  for(i in 1:nvar){
    indx<- factor(as.numeric(cut2(x[,i], g=quants)))
    dumx[,((quants*i)-(quants-1)):(quants*i)]<-model.matrix(~indx-1)
  }

Create treatment variable where half of the observations are treatment half are control

In [ ]:
 Treatment<-sample(rep(0:1,N/2))

Create true y. Note:
* True effect of Treatment on y is 1
* Most x variables have no effect on y
* Some x variables have nonlinear effects on y

In [ ]:
  y <- Treatment+x[,1] * 2+ x[,2] * 2.5 + 600*x[,3]-20*x[,3]^2-20*x[,3]^3+
                #3*exp(x[,4]+x[,10])+2*log(x[,6]+x[,10]+100)+ 
                600*x[,7]^2+30*x[,8]^2- 20*x[,8]^3+
                300*dumx[,(quants*5)]+90*dumx[,(quants*5)+1]+30*dumx[,(quants*5)+2]+
                100*dumx[,(quants*8)+1]+50*dumx[,(quants*8)+2]+70*dumx[,(quants*8)+3]+
                10*dumx[,(quants*9)]+60*dumx[,(quants*9)+3]-6*dumx[,(quants*9)+5]+
                rnorm(N)

## Step 3: Analyze Data

Estimate naive OLS model using only x variables and summarize

In [ ]:
fit.lm <- lm(y~Treatment+x)
summary(fit.lm)

Estimate naive Double Selection model using only x variables and summarize

In [ ]:
fit.ds <- rlassoEffect(x, y, Treatment, method = "double selection")
summary(fit.ds)

Now assume some x variables have polynomial effects on y and estimate OLS with x and polyx variables

In [ ]:
fit.lm.os <- lm(y~Treatment+x+polyx)
  summary(fit.lm.os)

Assume some x variables have binned effects on y and estimate OLS 

In [ ]:
fit.lm.os <- lm(y~Treatment+x+dumx+polyx)
  summary(fit.lm.os)

Assume some x variables have binned and polynomial effects on y and estimate double selection

In [ ]:
xds<-cbind(x,dumx,polyx)
  
fit.ds.os <- rlassoEffect(xds, y, Treatment, method = "double selection")
summary(fit.ds.os)

# Step 4: Monte Carlo

Each of these method give much different coefficients for Treatment. We will use a Monte Carlo to visualize the distribution of coefficients and measure the type 1 and type 2 errors for the tests.

### Create output matrices
Prefixes:
* betahat~ -- estimates of Treatment coefficient
* type1~ -- type one error(1), no type one error(0)
* type2~ -- type two error(1), no type two error(0), where false null is less than or equal to zero 

Suffixes:
* ~ols -- OLS estimation naive model
* ~ols.os -- OLS estimation overspecified model
* ~ds -- DS estimation naive model
* ~ds.os -- DS estimation overspecified model

In [ ]:
betahat.ols <- matrix(NA,nrow=n.sims,ncol=3)
betahat.ds <- matrix(NA,nrow=n.sims,ncol=1)
betahat.ols.os <- matrix(NA,nrow=n.sims,ncol=3)
betahat.ds.os <- matrix(NA,nrow=n.sims,ncol=1)
type1.ols <- matrix(NA,nrow=n.sims,ncol=1)
type1.ds <- matrix(NA,nrow=n.sims,ncol=1)
type1.ols.os <- matrix(NA,nrow=n.sims,ncol=1)
type1.ds.os <- matrix(NA,nrow=n.sims,ncol=1)
type2.ols <- matrix(NA,nrow=n.sims,ncol=1)
type2.ds <- matrix(NA,nrow=n.sims,ncol=1)
type2.ols.os <- matrix(NA,nrow=n.sims,ncol=1)
type2.ds.os <- matrix(NA,nrow=n.sims,ncol=1)


##### Takes 15-20 minutes to run
* Run the Monte Carlo experiment using previously described data generating process

* Calculate type 1 and type 2 errors

In [ ]:

for(j in 1:n.sims){
# Create data  
  x <- matrix(runif(N*50), nrow = N, ncol = 50)
  Treatment<-sample(rep(0:1,N/2))
  polyx<-cbind(x^2,x^3)
  nvar<-20
    quants<-5
    dumx<-matrix(nrow=N,ncol=5*nvar)
  for(i in 1:nvar){
    indx<- factor(as.numeric(cut2(x[,i], g=quants)))
    dumx[,((quants*i)-(quants-1)):(quants*i)]<-model.matrix(~indx-1)
  }
# Create y    
  y <- Treatment+x[,1] * 2+ x[,2] * 2.5 + 600*x[,3]-20*x[,3]^2-20*x[,3]^3+
                #3*exp(x[,4]+x[,10])+2*log(x[,6]+x[,10]+100)+ 
                600*x[,7]^2+30*x[,8]^2- 20*x[,8]^3+
                300*dumx[,(quants*5)]+90*dumx[,(quants*5)+1]+30*dumx[,(quants*5)+2]+
                100*dumx[,(quants*8)+1]+50*dumx[,(quants*8)+2]+70*dumx[,(quants*8)+3]+
                10*dumx[,(quants*9)]+60*dumx[,(quants*9)+3]-6*dumx[,(quants*9)+5]+
                rnorm(N)
#### Estimating true relationships
# Estimate naive OLS  
  fit.lm <- lm(y~Treatment+x)
  summary(fit.lm)
  
  b<-coef(summary(fit.lm))[,"Estimate"][2]
  s<-coef(summary(fit.lm))[,"Std. Error"][2]
  p<-coef(summary(fit.lm))[,"Pr(>|t|)"][2]
   
  t<-(b-1)/(s/sqrt(N))
# Store relavant statistics
  betahat.ols[j,]<-b
  
  if(t>1.96){
    type1.ols[j,1]<-1
  }
  else{
    type1.ols[j,1]<-0
  }
  
  if(b<0 | p>.05){
    type2.ols[j,1]<-1
  }
  else{
    type2.ols[j,1]<-0
  }

# Estimate naive DS  
  fit.ds <- rlassoEffect(x, y, Treatment, method = "double selection")
  summary(fit.ds)
  
  b.ds<-coef(summary(fit.ds))[,"Estimate."][1]
  s.ds<-coef(summary(fit.ds))[,"Std. Error"][1]
  p.ds<-coef(summary(fit.ds))[,"Pr(>|t|)"][1]
  
  t.ds<-(b.ds-1)/(s.ds/sqrt(N))
# Store relavant statistics  
  betahat.ds[j,]<-b.ds
  
  if(t.ds>1.96){
    type1.ds[j,1]<-1
  }
  else{
    type1.ds[j,1]<-0
  }
  if(b.ds<0 | p.ds>.05){
    type2.ds[j,1]<-1
  }
  else{
    type2.ds[j,1]<-0
  }
 
## Overspecify
# Estimate overspecified OLS 
  fit.lm.os <- lm(y~Treatment+x+dumx)
  summary(fit.lm.os)
  
  
  b.os<-coef(summary(fit.lm.os))[,"Estimate"][2]
  s.os<-coef(summary(fit.lm.os))[,"Std. Error"][2]
  p.os<-coef(summary(fit.lm.os))[,"Pr(>|t|)"][2]
  
  t.os<-(b.os-1)/(s.os/sqrt(N))
# Store relavant statistics  
  betahat.ols.os[j,]<-b.os
  
  if(t.os>1.96){
    type1.ols.os[j,1]<-1
  }
  else{
    type1.ols.os[j,1]<-0
  }
  
  if(b.os<0 | p.os>.05){
    type2.ols.os[j,1]<-1
  }
  else{
    type2.ols.os[j,1]<-0
  }

# Estimate overspecified DS
  xds<-cbind(x,dumx,polyx)
  
  fit.ds.os <- rlassoEffect(xds, y, Treatment, method = "double selection")
  summary(fit.ds.os)
  
  b.ds.os<-coef(summary(fit.ds.os))[,"Estimate."][1]
  s.ds.os<-coef(summary(fit.ds.os))[,"Std. Error"][1]
  p.ds.os<-coef(summary(fit.ds.os))[,"Pr(>|t|)"][1]
  
  t.ds.os<-(b.ds.os-1)/(s.ds.os/sqrt(N))
# Store relavant statistics  
  betahat.ds.os[j,]<-b.ds.os
  
  if(t.ds.os>1.96){
    type1.ds.os[j,1]<-1
  }
  else{
    type1.ds.os[j,1]<-0
  }
  
  if(b.ds.os<0 | p.ds.os>.05){
    type2.ds.os[j,1]<-1
  }
  else{
    type2.ds.os[j,1]<-0
  }
  print(paste0(j, ' of ',n.sims))
}

# Print type 1 and type 2 errors
print(paste0('type 1 error naive OLS rate = ',colMeans(type1.ols)))
print(paste0('type 1 error naive DS rate = ',colMeans(type1.ds)))
print(paste0('type 1 error overspecified OLS rate = ',colMeans(type1.ols.os)))
print(paste0('type 1 error overspecified DS rate = ',colMeans(type1.ds.os)))

print(paste0('type 2 error naive OLS rate = ',colMeans(type2.ols)))
print(paste0('type 2 error naive DS rate = ',colMeans(type2.ds)))
print(paste0('type 2 error overspecified OLS rate = ',colMeans(type2.ols.os)))
print(paste0('type 2 error overspecified DS rate = ',colMeans(type2.ds.os)))



Save density plots of coefficients

In [ ]:
png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecDS.png'))

plot (density(betahat.ds),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of naive Double Selection")

dev.off()

png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecOLS.png'))

plot (density(betahat.ols),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of naive OLS")

dev.off()


png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecDSos.png'))

plot (density(betahat.ds.os),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of overspecified Double Selection")

dev.off()

png(filename=paste(file.path(path.expand('~'),'outputs/Lab6/'),'MisspecOLSos.png'))

plot (density(betahat.ols.os),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of overspecified OLS")

dev.off()

### Question 1

Which the four methods has the best distribution? Why? What assumptions of OLS and DS are violated in the naive models? 

### Question 2

Why does the overspecified OLS have a lower type 1 error rate? 

### Question 3

The type 2 error rate is calculated incorrectly. Describe how would you correct it? 

### Question 4

Create a correlation between the Treatment and the independent variables. How does this change the OLS and DS coefficents? What other methods can be used to deal with this specification?

### Question 5

Change the model or data generating process so that an assumption of double selection is violated. How does this affect the ability of the model to draw valid inference?